# EJEMPLO

## Paso 1: Importar librerías 

In [ ]:
import numpy as np
import tensorflow as tf
from pathlib import Path
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

2024-12-11 23:14:20.656567: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Paso 2: Preparación de los datos
Vamos a cargar el conjunto de datos de viviendas de California en la variable housing. 

La función fetch_california_housing es parte de scikit-learn y se utiliza para cargar el conjunto de datos de viviendas de California. Este conjunto de datos contiene información sobre viviendas en diferentes distritos censales de California. La tarea común asociada con este conjunto de datos es predecir el valor medio de las viviendas para cada distrito censal.

In [2]:
housing = fetch_california_housing()

## Paso 3: División de los datos

In [3]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

## Paso 4: Creación del modelo

Construimos un modelo secuencial:
+ Capa de normalización
+ 3 capas densas de 50 neuronas con activación relu
+ 1 capa densa de 1 neurona sin función de activación ya que estamos ante un modelo de regresión

In [4]:
tf.random.set_seed(42)

# Esta capa realiza normalización por lotes (batch normalization) en los datos de entrada
norm_layer = tf.keras.layers.Normalization(input_shape=X_train.shape[1:])

# Se crea el modelo con la capa de normalización y 4 capas densas (fully connected) con 50 neuronas cada una y función de activación ReLU (Rectified Linear Unit) y una capa de salida con una neurona y sin función de activación
model = tf.keras.Sequential([
    norm_layer,
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1)
])

# se define el optimizador Adam con una tasa de aprendizaje de 0.001
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

# Se compila el modelo con el optimizador Adam y la función de pérdida MSE (Mean Squared Error) y la métrica RMSE (Root Mean Squared Error)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

# el metodo adapt de la capa de normalización calcula la media y la desviación estándar de cada característica y las almacena en la capa de normalización
norm_layer.adapt(X_train)


/opt/anaconda3/envs/sapa/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [5]:
# calcular el valor máximo en el conjunto de entrenamiento de las etiquetas (y_train). 
np.max(y_train)

5.00001

## Paso 5: Entrenamiento del modelo

In [ ]:
# Se entrena el modelo con 20 épocas y se especifica el conjunto de validación para evaluar el modelo en cada época y evitar el sobreajuste 
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_valid, y_valid))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - RootMeanSquaredError: 1.1236 - loss: 1.3380 - val_RootMeanSquaredError: 0.6161 - val_loss: 0.3796
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6329 - loss: 0.4011 - val_RootMeanSquaredError: 0.5885 - val_loss: 0.3463
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6048 - loss: 0.3660 - val_RootMeanSquaredError: 0.5901 - val_loss: 0.3482
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.5931 - loss: 0.3520 - val_RootMeanSquaredError: 0.6508 - val_loss: 0.4235
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - RootMeanSquaredError: 0.5856 - loss: 0.3431 - val_RootMeanSquaredError: 0.8703 - val_loss: 0.7574
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5788 - loss: 0.3352 - val_RootMeanSquaredError: 1.0044 - val_loss: 1.0088
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5717 - los

## Paso 6 y 7: Evaluamos y generamos predicciones

In [7]:
# Se evalúa el modelo en el conjunto de prueba y se calcula el error cuadrático medio (MSE) y la raíz del error cuadrático medio (RMSE) 
mse_test, rmse_test = model.evaluate(X_test, y_test)
# se crea una nueva variable X_new que contiene las primeras tres filas (o ejemplos) del conjunto de prueba 
X_new = X_test[:3]
# y con ella se realizan predicciones con el modelo entrenado 
y_pred = model.predict(X_new)

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 811us/step - RootMeanSquaredError: 0.5310 - loss: 0.2822
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


In [8]:
rmse_test

0.539259672164917

El valor de RMSE indica en términos generales cuánto se desvían en promedio las predicciones del modelo de los valores reales.

In [9]:
y_pred

array([[0.48103914],
       [0.7894206 ],
       [4.9853277 ]], dtype=float32)

## Paso 8: Guardar y restaurar el modelo

Guardamos el modelo

In [28]:
model.save("Resultados/my_keras_model.keras")

Restauramos el modelo

In [17]:
model1 = tf.keras.models.load_model("Resultados/my_keras_model.keras")
X_new = X_test[3:7]
y_pred = model1.predict(X_new)
y_pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[2.6051292],
       [2.8231306],
       [1.7275032],
       [2.1425018]], dtype=float32)

También se pueden guardar y restaurar los pesos

In [23]:
model.save_weights("Resultados/my_weights.weights.h5")

In [25]:
model.load_weights("Resultados/my_weights.weights.h5")

## Paso 9: Retrollamadas

ModelCheckpoint: este objeto se utiliza como un callback durante el entrenamiento de un modelo para guardar los pesos del modelo en un punto determinado.
Parametros:
+ "Resultados/my_checkpoints": es la ruta al archivo o directorio donde se guardarán los pesos del modelo.
+ save_weights_only=True: este parámetro especifica que solo se deben guardar los pesos del modelo, no el modelo completo.

In [29]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint( "Resultados/my_checkpoints.weights.h5", save_weights_only=True)

Durante el entrenamiento, el callback ModelCheckpoint verifica si hay una mejora en la métrica de validación monitorizada y guarda los pesos del modelo si se cumple la condición. 

Después del entrenamiento, puedes cargar los pesos del mejor modelo utilizando el archivo guardado para realizar predicciones en nuevos datos.

In [30]:
history = model.fit( X_train, y_train, epochs=40, validation_data=(X_valid, y_valid),callbacks=[checkpoint_cb],)

Epoch 1/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.5222 - loss: 0.2727 - val_RootMeanSquaredError: 0.9828 - val_loss: 0.9659
Epoch 2/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5207 - loss: 0.2711 - val_RootMeanSquaredError: 0.6369 - val_loss: 0.4056
Epoch 3/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5181 - loss: 0.2685 - val_RootMeanSquaredError: 0.7151 - val_loss: 0.5114
Epoch 4/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.5156 - loss: 0.2659 - val_RootMeanSquaredError: 0.5468 - val_loss: 0.2990
Epoch 5/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5139 - loss: 0.2642 - val_RootMeanSquaredError: 0.6106 - val_loss: 0.3728
Epoch 6/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5115 - loss: 0.2617 - val_RootMeanSquaredError: 0.5273 - val_loss: 0.2780
Epoch 7/40
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.5097 - los

Se incorpora un nuevo callback llamado EarlyStopping durante el entrenamiento del modelo. 
+ tf.keras.callbacks.EarlyStopping: es la clase que detiene el entrenamiento del modelo cuando una métrica monitorizada deja de mejorar.

+ patience: este parámetro indica la cantidad de épocas para esperar antes de detener el entrenamiento después de que la métrica monitorizada haya dejado de mejorar.

+ restore_best_weights=True: este parámetro especifica si restaurar los pesos del modelo a los mejores obtenidos durante el entrenamiento cuando se detiene. Al establecerlo en True, aseguras que los pesos del modelo se vuelvan a cargar con los mejores obtenidos en lugar de los de la última época.

In [31]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)
history = model.fit(
    X_train,
    y_train,
    epochs=100,
    validation_data=(X_valid, y_valid),
    callbacks=[checkpoint_cb, early_stopping_cb],
)


Epoch 1/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.4690 - loss: 0.2200 - val_RootMeanSquaredError: 1.9746 - val_loss: 3.8992
Epoch 2/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.4845 - loss: 0.2352 - val_RootMeanSquaredError: 0.5130 - val_loss: 0.2632
Epoch 3/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.4735 - loss: 0.2243 - val_RootMeanSquaredError: 0.6878 - val_loss: 0.4730
Epoch 4/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.4689 - loss: 0.2199 - val_RootMeanSquaredError: 0.7236 - val_loss: 0.5235
Epoch 5/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.4662 - loss: 0.2174 - val_RootMeanSquaredError: 0.5405 - val_loss: 0.2921
Epoch 6/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 0.4621 - loss: 0.2136 - val_RootMeanSquaredError: 0.7138 - val_loss: 0.5095
Epoch 7/100
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.461